# **Smart Transportation**

In [149]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load the data
data = pd.read_csv('/content/smartcity_data.csv')

# Preprocessing
# Convert timestamp to datetime
print("Converting 'timestamp' to datetime format...")
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Add temporal features
data['hour'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.dayofweek

# Feature engineering for congestion prediction
data['congestion_level'] = 100 - data['speed']

# Select features for both tasks
features = ['temperature', 'precipitation', 'windSpeed', 'humidity', 'airQualityIndex', 'hour', 'day_of_week']

# --- TRAFFIC CONGESTION PREDICTION ---
print("Preparing data for Traffic Congestion Prediction...")
X_congestion = data[features]
y_congestion = data['congestion_level']

# Normalize features
scaler = StandardScaler()
X_congestion = scaler.fit_transform(X_congestion)

# Split into train and test sets
X_train_cong, X_test_cong, y_train_cong, y_test_cong = train_test_split(X_congestion, y_congestion, test_size=0.3, random_state=42)

# Train Random Forest Regressor
print("Training Random Forest Regressor...")
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train_cong, y_train_cong)


# Predict and Evaluate for Traffic Congestion
print("Evaluating the Regressor...")
y_pred_cong = rf_regressor.predict(X_test_cong)

# --- INCIDENT PREDICTION ---
print("Preparing data for Incident Prediction...")
X_incident = data[features]
y_incident = data['type']

# Normalize features
X_incident = scaler.fit_transform(X_incident)

# Handle class imbalance using SMOTE
print("Balancing the dataset using SMOTE...")
smote = SMOTE(random_state=42)
X_incident_balanced, y_incident_balanced = smote.fit_resample(X_incident, y_incident)

# Split into train and test sets
X_train_inc, X_test_inc, y_train_inc, y_test_inc = train_test_split(X_incident_balanced, y_incident_balanced, test_size=0.3, random_state=42)

# Train Random Forest Classifier
print("Training Random Forest Classifier...")
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_inc, y_train_inc)

# Predict and Evaluate for Incidents

y_pred_inc = rf_classifier.predict(X_test_inc)



Converting 'timestamp' to datetime format...
Preparing data for Traffic Congestion Prediction...
Training Random Forest Regressor...
Evaluating the Regressor...
Preparing data for Incident Prediction...
Balancing the dataset using SMOTE...
Training Random Forest Classifier...


In [150]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels
label_encoder = LabelEncoder()
data['type_encoded'] = label_encoder.fit_transform(data['type'])

# Check the mapping
print("Label Encoding Mapping:")
for index, class_name in enumerate(label_encoder.classes_):
    print(f"{index}: {class_name}")

# Replace the target variable with the encoded version
y_incident = data['type_encoded']


Label Encoding Mapping:
0: Accident
1: Crime
2: Fire
3: Medical


In [151]:
# Balance the dataset using SMOTE
X_incident_balanced, y_incident_balanced = smote.fit_resample(X_incident, y_incident)


In [152]:
X_train_inc, X_test_inc, y_train_inc, y_test_inc = train_test_split(
    X_incident_balanced, y_incident_balanced, test_size=0.3, random_state=42
)


In [153]:
!pip install catboost


In [154]:
from catboost import CatBoostClassifier

catboost_classifier = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=10, random_state=42)
catboost_classifier.fit(X_train_inc, y_train_inc)
y_pred_inc = catboost_classifier.predict(X_test_inc)


0:	learn: 1.3801237	total: 280ms	remaining: 55.7s
1:	learn: 1.3748488	total: 675ms	remaining: 1m 6s
2:	learn: 1.3701954	total: 786ms	remaining: 51.6s
3:	learn: 1.3682811	total: 793ms	remaining: 38.9s
4:	learn: 1.3653357	total: 826ms	remaining: 32.2s
5:	learn: 1.3616734	total: 898ms	remaining: 29s
6:	learn: 1.3578478	total: 990ms	remaining: 27.3s
7:	learn: 1.3559226	total: 1s	remaining: 24.1s
8:	learn: 1.3519190	total: 1.18s	remaining: 25s
9:	learn: 1.3483610	total: 1.27s	remaining: 24.2s
10:	learn: 1.3458948	total: 1.51s	remaining: 26s
11:	learn: 1.3423890	total: 2.08s	remaining: 32.6s
12:	learn: 1.3385702	total: 2.51s	remaining: 36.1s
13:	learn: 1.3354180	total: 3.07s	remaining: 40.8s
14:	learn: 1.3326118	total: 3.82s	remaining: 47.1s
15:	learn: 1.3298380	total: 4.12s	remaining: 47.4s
16:	learn: 1.3295194	total: 4.14s	remaining: 44.6s
17:	learn: 1.3270452	total: 4.92s	remaining: 49.8s
18:	learn: 1.3252856	total: 5.63s	remaining: 53.7s
19:	learn: 1.3232658	total: 6.22s	remaining: 56s
2

In [155]:
from sklearn.metrics import classification_report

# Generate classification report
print("Classification Report:")
print(classification_report(y_test_inc, y_pred_inc, target_names=label_encoder.classes_))


Classification Report:
              precision    recall  f1-score   support

    Accident       0.45      0.05      0.09        98
       Crime       0.31      0.61      0.41        84
        Fire       0.42      0.16      0.24        79
     Medical       0.26      0.47      0.34        73

    accuracy                           0.31       334
   macro avg       0.36      0.32      0.27       334
weighted avg       0.37      0.31      0.26       334



In [156]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=10, random_state=42)
xgb_classifier.fit(X_train_inc, y_train_inc)

# Predict on the test set
y_pred_inc = xgb_classifier.predict(X_test_inc)


In [157]:
from sklearn.metrics import classification_report

print("Classification Report:")
print(classification_report(
    label_encoder.inverse_transform(y_test_inc),
    label_encoder.inverse_transform(y_pred_inc),
    target_names=label_encoder.classes_
))



Classification Report:
              precision    recall  f1-score   support

    Accident       0.45      0.05      0.09        98
       Crime       0.30      0.58      0.40        84
        Fire       0.40      0.18      0.25        79
     Medical       0.26      0.45      0.33        73

    accuracy                           0.30       334
   macro avg       0.35      0.32      0.27       334
weighted avg       0.36      0.30      0.26       334



In [158]:
!pip install dash

In [159]:
print(data.head())
print(f"y_test_cong: {y_test_cong[:5]}")
print(f"y_pred_cong: {y_pred_cong[:5]}")


  entry_id deviceId           timestamp   location       speed direction  \
0     V001    D5295 2024-12-02 15:15:00  Riverhead   67.480511      West   
1     V002    D9697 2024-12-02 15:13:00  Smithtown  111.758549     South   
2     V003    D4534 2024-12-02 15:18:00    Babylon   63.901989      West   
3     V004    D8364 2024-12-02 14:32:00  Riverhead   52.845071      East   
4     V005    D3016 2024-12-02 14:13:00  Smithtown   91.319236      East   

        make   model    year  fuelType  ...      type    status  \
0  Chevrolet   Camry  2023.0    Diesel  ...      Fire   Ongoing   
1  Chevrolet  Accord  2010.0    Hybrid  ...  Accident  Resolved   
2  Chevrolet  Altima  2011.0  Gasoline  ...     Crime  Resolved   
3      Honda  Accord  2014.0  Electric  ...      Fire   Ongoing   
4       Ford   Focus  2021.0    Diesel  ...   Medical  Resolved   

               description    id   latitude  longitude  hour  day_of_week  \
0   Description of Medical  G061  40.917041 -72.662041    15   

In [160]:
import pandas as pd

# Load the CSV file
file_path = '/content/smartcity_data.csv'

# Specify the column name
column_name = 'vehicleType'
# Get unique values as a set
unique_values = set(data[column_name])

# Display the unique values
print(f"Unique values in '{column_name}':", unique_values)

Unique values in 'vehicleType': {'SUV', 'Sedan', 'Truck', 'Van', 'Motorcycle'}


In [161]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset
data = pd.read_csv('/content/smartcity_data.csv')

# Preprocessing
# Convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')
data.dropna(subset=['timestamp', 'latitude', 'longitude', 'speed'], inplace=True)
data['date'] = data['timestamp'].dt.date

# Feature Engineering
print("Adding temporal and congestion features...")
data['hour'] = data['timestamp'].dt.hour
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['congestion'] = 100 - data['speed']  # Assuming congestion inversely relates to speed

# Align columns for dashboard
traffic_data = data[['latitude', 'longitude', 'speed', 'timestamp', 'date', 'congestion']].copy()
weather_data = data[['latitude', 'longitude', 'airQualityIndex', 'precipitation']].copy()
emergency_data = data[['latitude', 'longitude', 'type', 'timestamp', 'date']].copy()


Adding temporal and congestion features...


In [162]:
traffic_data.head()


,latitude,longitude,speed,timestamp,date,congestion
0,40.917041,-72.662041,67.480511,2024-12-02 15:15:00,2024-12-02,32.519489
1,40.855933,-73.200668,111.758549,2024-12-02 15:13:00,2024-12-02,-11.758549
2,40.703950,-73.377679,63.901989,2024-12-02 15:18:00,2024-12-02,36.098011
3,40.917041,-72.662041,52.845071,2024-12-02 14:32:00,2024-12-02,47.154929
4,40.855933,-73.200668,91.319236,2024-12-02 14:13:00,2024-12-02,8.680764


In [163]:
weather_data.head()

,latitude,longitude,airQualityIndex,precipitation
0,40.917041,-72.662041,149.725929,25.449616
1,40.855933,-73.200668,149.725929,25.449616
2,40.703950,-73.377679,149.725929,25.449616
3,40.917041,-72.662041,149.725929,25.449616
4,40.855933,-73.200668,149.725929,25.449616


In [164]:
emergency_data.head()

,latitude,longitude,type,timestamp,date
0,40.917041,-72.662041,Fire,2024-12-02 15:15:00,2024-12-02
1,40.855933,-73.200668,Accident,2024-12-02 15:13:00,2024-12-02
2,40.703950,-73.377679,Crime,2024-12-02 15:18:00,2024-12-02
3,40.917041,-72.662041,Fire,2024-12-02 14:32:00,2024-12-02
4,40.855933,-73.200668,Medical,2024-12-02 14:13:00,2024-12-02


In [165]:

# Initialize Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div(style={'backgroundColor': '#f5f5f5', 'color': '#000'}, children=[
    html.Div([
        html.H1("Smart Traffic Dashboard", style={'textAlign': 'center', 'color': '#333'}),
    ], style={'backgroundColor': '#ffffff', 'padding': '10px'}),

    html.Div([
        html.Label("Select Vehicle Type:", style={'color': '#333'}),
        dcc.Dropdown(
            id='vehicle-type-dropdown',
            options=[
              {'label': 'SUV', 'value': 'SUV'},
                {'label': 'Sedan', 'value': 'Sedan'},
                {'label': 'Truck', 'value': 'Truck'},
                {'label': 'Van', 'value': 'Van'},
                {'label': 'Motorcycle', 'value': 'Motorcycle'}
            ],
            value='Car',
            style={'width': '50%'}
        )
    ], style={'padding': '10px', 'backgroundColor': '#ffffff'}),

    html.Div([
        html.Button('Get Travel Recommendations', id='recommendation-button', n_clicks=0)
    ], style={'padding': '10px'}),

    html.Div(id='recommendations-output', style={'padding': '10px', 'backgroundColor': '#e0e0e0'}),

    html.Div([
        dcc.Graph(id='traffic-hotspot-map', config={'displayModeBar': False}),
        dcc.Graph(id='congestion-prediction-chart', config={'displayModeBar': False})
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '10px'}),

    html.Div([
        dcc.Graph(id='air-quality-map', config={'displayModeBar': False}),
        dcc.Graph(id='incident-trends-chart', config={'displayModeBar': False})
    ], style={'display': 'flex', 'justifyContent': 'space-between', 'padding': '10px'}),
])

# Callbacks
@app.callback(
    [
        Output('traffic-hotspot-map', 'figure'),
        Output('congestion-prediction-chart', 'figure'),
        Output('air-quality-map', 'figure'),
        Output('incident-trends-chart', 'figure'),
        Output('recommendations-output', 'children')
    ],
    [
        Input('vehicle-type-dropdown', 'value'),
        Input('recommendation-button', 'n_clicks')
    ]
)
def update_dashboard(vehicle_type, n_clicks):
    try:
        # Traffic Hotspot Map
        hotspot_fig = px.density_mapbox(
            traffic_data, lat='latitude', lon='longitude', z='congestion',
            radius=15, center=dict(lat=40.7, lon=-73.9), zoom=10,
            mapbox_style='open-street-map', title='Traffic Hotspots'
        )

        # Congestion Prediction
        traffic_data_sorted = traffic_data.sort_values('timestamp')
        traffic_data_sorted['moving_avg'] = traffic_data_sorted['congestion'].rolling(window=5).mean()
        congestion_fig = px.line(
            traffic_data_sorted, x='timestamp', y='moving_avg',
            title='Congestion Prediction',
            labels={'timestamp': 'Time', 'moving_avg': 'Predicted Congestion Level'}
        )

        # Air Quality Map
        air_quality_fig = px.scatter_mapbox(
            weather_data, lat='latitude', lon='longitude', size='airQualityIndex',
            color='airQualityIndex', title='Air Quality Levels',
            mapbox_style='carto-darkmatter'
        )

        # Incident Trends by Type
        incident_trends = emergency_data.groupby(['type', 'date']).size().reset_index(name='count')
        incident_trends_fig = px.bar(
            incident_trends, x='date', y='count', color='type',
            title='Incident Trends by Type',
            labels={'date': 'Date', 'count': 'Incident Count', 'type': 'Incident Type'}
        )

        # Dynamic Recommendations
        recommendations = [
            "**Ideal time to start your journey**: 2024-12-04 16:25:09",
            "**Estimated fuel consumption**: 0.98 liters for 10 km"
        ]
        recommendations_html = [html.P(rec) for rec in recommendations]

        return hotspot_fig, congestion_fig, air_quality_fig, incident_trends_fig, recommendations_html

    except Exception as e:
        print(f"Error in callback: {e}")
        return (
            go.Figure(), go.Figure(), go.Figure(), go.Figure(),
            [html.P("Error generating recommendations.")]
        )

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>